# PROMISE12 prostate segmentation demo

## Preparation:
1) Make sure you have setup up the PROMISE12 data set. If not, see data/PROMISE12/README.md for instructions.

2) Make sure you are in NiftyNet root, setting niftynet_path correctly to the path with the niftynet folder in it

In [1]:
import os,sys 
niftynet_path=r'path/to/NiftyNet'
os.chdir(niftynet_path)

3) Make sure you have all the dependencies installed (replacing gpu with cpu for cpu-only mode):

In [ ]:
import pip
pip.main(['install','-r','requirements-gpu.txt'])

## Training a network from the command line
The simplest way to use NiftyNet is via the commandline net_segmentation.py script. Normally, this is done on the command line with a command like this from the NiftyNet root directory:

```python net_segmentation.py train --conf demo/PROMISE12/promise12_demo_train_config.txt --image_size 32 --label_size 32 --max_iter 10```

Notice that we use configuration file that is specific to this experiment. This file contains default settings. Also note that we can override these settings on the command line.

To execute NiftyNet from within the notebook, you can run the following python code:  

In [ ]:
import niftynet
sys.argv=['','train', '--conf',os.path.join('demos','PROMISE12','promise12_demo_train_config.txt'),'--image_size','32','--label_size','32','--max_iter','10']
niftynet.main()

Now you have trained (a few iterations of) a deep learning network for medical image segmentation. If you have some time on your hands, you can finish training the network (by leaving off the max_iter argument) and try it out, by running the following command

```python net_segmentation.py inference --conf demo/PROMISE12/promise12_demo_inference_config.txt --image_size 32 --label_size 32```

or the following python code in the Notebook

In [ ]:
import niftynet
sys.argv=['', 'inference','--conf',os.path.join('demos','PROMISE12','promise12_demo_train_config.txt'),'--image_size','32','--label_size','32']
niftynet.main()

Otherwise, you can load up some pre-trained weights for the network:

```python net_segmentation.py inference --conf demo/PROMISE12/promise12_demo_config.txt --model_dir demo/PROMISE12/pretrained```
or the following python code in the Notebook

In [ ]:
import niftynet
sys.argv=['', 'inference', '--conf',os.path.join('demos','PROMISE12','promise12_demo_inference_config.txt'), '--model_dir', os.path.join('demos','PROMISE12','pretrained')]
niftynet.main()

You can find your segmented images in output/promise12_demo

NiftyNet has taken care of a lot of details behind the scenes:
1. Organizing data into a dataset of images and segmentation labels
2. Building a deep leaning network (in this case, it is based on VNet by Milletari et al.)
3. Added deep learning infrastruture, such as a loss function for segmentation, the ADAM optimizer.
4. Added augmentation, where the images are zoomed and rotated a little bit for every training step so that you do not over-fit the data
5. Run the training algorithm

All of this was controlled by the configuration file.

## The configuration file

 Let's take a closer look at the configuration file. Further details about the configuration settings are available in ```config/readme.md```

These lines define how NiftyNet organizes your data. In this case, in the ./data/PROMISE12 folder there is one T2-weighted MR image named 'Case??_T2.nii.gz' and one reference segmentation named 'Case??_segmentation.nii.gz' per patient. The images for each patient are automatically grouped because they share the same prefix 'Case??'. For training, we exclude patients Case20-Case26, and for inference, we only include patients Case20-Case26, so that our training and inference data are mutually exclusive.

These lines are setting up some system parameters: which GPUs to use (in this case whatever is available), where to save the trained network parameters, how many images to queue up in advance so that the GPU isn't waiting for data, and how many threads to use for queuing them up.

These lines are setting parameters for the network itself.

These lines are setting parameters for preprocessing and data augmentation. 

These lines are setting parameters for the inference algorithm

## Run the demo interactively
To see the inner workings of NiftyNet, we can run this demo within the Notebook interactively. A simple working example, (with many NiftyNet features left out) follows.

Begin by importing libraries, including tensorflow, core NiftyNet libraries and helper functions:

In [ ]:
import tensorflow as tf
from network.vnet import VNet # This module is the network architecture we will use for segmentation
from utilities.csv_table import CSVTable,SubjectTable # This class holds the subject ids and file names
from engine.volume_loader import VolumeLoaderLayer # This module loads sets of images to feed into the network
import engine.training
import utilities.parse_user_params as parse_user_params

You can use helper functions to parse commandline parameters and automatically match patient data (which is what net_segmentation.py did).

In [ ]:
sys.argv=['','train', '--conf',os.path.join('demos','PROMISE12','promise12_demo_train_config.txt'),'--image_size','32','--label_size','32','--max_iter','10']
param, csv_dict = parse_user_params.run()
csv_loader = CSVTable(csv_dict=csv_dict, allow_missing=True)

Or you can use write your own code to change parameters and specify the patients to use.

In [ ]:
params = parse_user_params.default_params('train','./demos/PROMISE12/promise12_demo_train_config.txt')
params.max_iter=10
loader = SubjectTable([['Case%02d'%i,[['./data/PROMISE12/Case%02d_T2.nii.gz'%i]],[['./data/PROMISE12/Case%02d_segmentation.nii.gz'%i]],'',''] for i in range(20)])

Note the form of the parameter passed to SubjectTable. It is a list, where each element represents one subject. Each element is a list where the first element is the id of the patient, the second element is a list of lists of image names representing the images being segmented. The list of lists is flexible enough to define multiple modalities and multiple time points if that suits your segmentation problem. The third element is a list of lists representing the segmentation labels. The fourth and fifth elements are blanks, but for some problems could represent a weighting image, and additional notes each subject. 

Finally, create the data loader and run the training algorithm

In [ ]:
volume_loader = VolumeLoaderLayer(loader)
engine.training.run(VNet, params, volume_loader, 'gpu')

This simple snippet already illustrates several key concepts in NiftyNet:
1) NiftyNet is built of various modules: pre-built networks, like VNet; the training module which executes the training; the VolumeLoader which loads images and structures them into minibatches. 
2) These components are parameterized by a common object, which can be read from configuration files and manipulated in the code.
## Summary

In this demo 
1. you learned to run training and testing for a deep-learning-based segmentation pipeline from the command-line and from python code directly; 
2. you also learned about the NiftyNet configuration files, and how they control the learning and inference process; and 
3. you learned multiple ways to tell NiftyNet which data to use.